## [[Discourse] AttributeError: 'DataFrame' object has no attribute 'repartition' #207](https://github.com/coiled/dask-community/issues/207)

User is trying to do: read data --> filter --> convert some cols to tensors

### Original with code with toy data

In [1]:
import dask
# import torch

import pandas as pd
import dask.dataframe as dd

In [2]:
df = pd.DataFrame({'userID': ['1', '1', '2', '3'], 'questionID': ['a', 'b', 'b', 'c'], 'result': ['99', '100', '98', '97']})
df

,userID,questionID,result
0,1,a,99
1,1,b,100
2,2,b,98
3,3,c,97


In [3]:
ddf = dd.from_pandas(df, npartitions=2)

In [5]:
MAX_LEN = 300
users = df.userID.unique()
paths = []
labels = []

In [6]:
# @dask.delayed(nout=2)
def get_paths(df, user):
    temp = df.loc[df.userID == user]
    npart = round(len(temp)/MAX_LEN)
    if npart == 0:
        return (temp.questionID.values, temp.result.values)
    else:
        parted_df = temp.repartition(npartitions=npart)
        return (parted_df.partitions[0].questionID.values,
                parted_df.partitions[0].result.values)

In [7]:
# @dask.delayed
def convert_to_tensor(x):
    # return torch.tensor(x)
    return x

In [8]:
for user in users:
    path, label = get_paths(df, user)
    paths.append(convert_to_tensor(path))
    labels.append(convert_to_tensor(label))

In [ ]:
dask.compute(*paths)

In [9]:
dask.compute(*labels)

In [9]:
paths

[array(['a', 'b'], dtype=object),
 array(['b'], dtype=object),
 array(['c'], dtype=object)]

In [10]:
labels

[array(['99', '100'], dtype=object),
 array(['98'], dtype=object),
 array(['97'], dtype=object)]

---

### User's updated code

In [ ]:
trunc_df = pd.DataFrame()
for user in tqdm(df.userID.unique()):
    user_df = df.loc[df.userID == user].iloc[-MAX_LEN:]
    trunc_df = pd.concat([trunc_df, user_df])

In [ ]:
@dask.delayed(nout=2)
def get_paths(df, user):
    temp = df.loc[df.userID == user]
    return (temp.questionID.values,
            temp.result.values)

---

### Our implementation with groupby

In [28]:
import numpy as np
import pandas as pd
import dask.dataframe as dd

df = pd.DataFrame({'userID': np.repeat(np.array([1,2,3]), 400),
                   'questionID': np.arange(0, 1200),
                   'result': np.arange(1200, 2400),
                  })

ddf = dd.from_pandas(df, npartitions=2)

ddf.groupby('userID').apply(lambda x: x[['questionID', 'result']].head(300)).compute()

/var/folders/n6/th0cntvx43x3mvr1f768tygm0000gn/T/ipykernel_63600/165589648.py:12: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf.groupby('userID').apply(lambda x: x[['questionID', 'result']].head(300)).compute()


questionID  result
userID                         
1      0              0    1200
       1              1    1201
       2              2    1202
       3              3    1203
       4              4    1204
...                 ...     ...
3      1095        1095    2295
       1096        1096    2296
       1097        1097    2297
       1098        1098    2298
       1099        1099    2299

[900 rows x 2 columns]